In [1]:
import numpy as np
import pandas as pd
import IPython.display as ipd
import matplotlib.pyplot as plt

import scipy 
import scipy.fft

import librosa
import librosa.display
import librosa.filters
import librosa.feature

import os
import sklearn

import pickle

%matplotlib inline

## Загружаю расчитанные фичи и словарь меток.

In [2]:
def pkl_load(file):
    return pickle.load(open(file,'rb'))

In [4]:
from sklearn.model_selection import train_test_split

data = pd.DataFrame(pkl_load('../audioset/Processed/features.pkl'))
actual_dict = pkl_load('../audioset/Processed/actual_dict.pkl')

## Разбиваю на тренировачную тестовую выборки.

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data['features'].values, data['label'].values, test_size=0.25)
np.asarray(x_test).shape

(83137,)

## Код для обучения Random forest

* Создание классификатора с задаными параметрами (кол-во деревьев, глубина деревьев).
* Перераспределение данных для использования на эпохах обучения.
* Обучение (заполнение модели, определение промежуточного результата).

In [7]:
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

def make_training_model(x, y, epoch_cnt, n_estimators, forest_depth):
    classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=forest_depth, n_jobs=-1)
    
    Selector = KFold(n_splits=epoch_cnt, random_state=None, shuffle=False)
    
    print('\n\n', classifier)
    
    Number = 0
    for train_index, test_index in Selector.split(X=x, y=y):   
        Number += 1

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        classifier.fit(x_train.tolist(), y_train.tolist())
        print('\nBlock', Number, ' score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))
        
    return classifier

Первый эксперимент. Попробовал взять кол-во деревьев равное кол-ву меток. И небольшую глубину деревьев.

In [10]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=4)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=4, n_estimators=61, n_jobs=-1)

Block 1  score: 0.26748139594559917

Block 2  score: 0.26379266102129845

Block 3  score: 0.2657172183731075

Block 4  score: 0.2670964844752374

Block 5  score: 0.26305491403643827

Block 6  score: 0.263632281241981

Block 7  score: 0.2644341801385681

Block 8  score: 0.26581344624069797

Result score: 0.2669930355918544


Увеличиваю глубину деревьев -> результаты улучшаются

In [11]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=6)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=6, n_estimators=61, n_jobs=-1)

Block 1  score: 0.2695984090325892

Block 2  score: 0.2659096741082884

Block 3  score: 0.2675134719014627

Block 4  score: 0.26924557351809086

Block 5  score: 0.26472286374133946

Block 6  score: 0.26539645881447266

Block 7  score: 0.2663587374903772

Block 8  score: 0.2678983833718245

Result score: 0.269579128426573


Немного увеличиваю глубину и значительно кол-во деревьев. -> результаты меняются не сильно.

In [13]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict)*4, forest_depth=10)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=10, n_estimators=244, n_jobs=-1)

Block 1  score: 0.2943610469591994

Block 2  score: 0.2945535026943803

Block 3  score: 0.2949384141647421

Block 4  score: 0.2959648447523736

Block 5  score: 0.2907364639466256

Block 6  score: 0.29217988196048245

Block 7  score: 0.2953874775468309

Block 8  score: 0.29484218629715164

Result score: 0.2970518541684208


Так как увеличение кол-ва деревьев ни к чему не привело. Пробую увеличить глубину -> результаты заметно лучше.

In [15]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=len(actual_dict), forest_depth=16)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=16, n_estimators=61, n_jobs=-1)

Block 1  score: 0.3695150115473441

Block 2  score: 0.3727226071336926

Block 3  score: 0.3719207082371055

Block 4  score: 0.37570567102899666

Block 5  score: 0.3706376700025661

Block 6  score: 0.3714395688991532

Block 7  score: 0.37570567102899666

Block 8  score: 0.3743264049268668

Result score: 0.3750676594055595


Значительно уменьшаю кол-во деревьев и сильно увеличевую глубину -> результаты немного лучше

In [17]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=10, forest_depth=24)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=24, n_estimators=10, n_jobs=-1)

Block 1  score: 0.3861303566846292

Block 2  score: 0.3943738773415448

Block 3  score: 0.3941172696946369

Block 4  score: 0.39463048498845266

Block 5  score: 0.38965871182961254

Block 6  score: 0.3922247882986913

Block 7  score: 0.39273800359250705

Block 8  score: 0.3923851680780087

Result score: 0.39170285191912146


Пытаюсь еще увеличить глубину. Результыты не меняются. Значит дальше будем менять другие два параметра.

In [18]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=10, forest_depth=64)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=64, n_estimators=10, n_jobs=-1)

Block 1  score: 0.3894341801385681

Block 2  score: 0.39216063638696436

Block 3  score: 0.3894341801385681

Block 4  score: 0.3954003079291763

Block 5  score: 0.39039645881447266

Block 6  score: 0.38917757249166024

Block 7  score: 0.39248139594559917

Block 8  score: 0.3884077495509366

Result score: 0.38933326918219324


Снова пробую увеличить кол-во деревьев и кол-во эпох -> результаты лучше еще на первой эпохе. Значит кол-во эпох не сильно повлияло на результат.

In [20]:
classifier = make_training_model(x_train, y_train, epoch_cnt=20, n_estimators=100, forest_depth=32)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=32, n_jobs=-1)

Block 1  score: 0.42875471092935613

Block 2  score: 0.43693368615187234

Block 3  score: 0.4421457782054366

Block 4  score: 0.44342875471092935

Block 5  score: 0.4351695934568198

Block 6  score: 0.43436773314088684

Block 7  score: 0.4451126613743886

Block 8  score: 0.44142410392109693

Block 9  score: 0.43416198877305534

Block 10  score: 0.43360064153969524

Block 11  score: 0.43544506816359263

Block 12  score: 0.44121892542101043

Block 13  score: 0.4299919807538091

Block 14  score: 0.44402566158781076

Block 15  score: 0.4347233360064154

Block 16  score: 0.44274258219727347

Block 17  score: 0.43905372894947875

Block 18  score: 0.4408981555733761

Block 19  score: 0.4372093023255814

Block 20  score: 0.44274258219727347

Result score: 0.4390584216413871


Итоговый эксперимент. Пробую еще увеличить кол-во деревьев -> изменений нет -> Значит с данными параметрами добились максимума.

In [21]:
classifier = make_training_model(x_train, y_train, epoch_cnt=8, n_estimators=150, forest_depth=32)
print('\nResult score:', metrics.accuracy_score(y_test.tolist(), classifier.predict(x_test.tolist())))



 RandomForestClassifier(max_depth=32, n_estimators=150, n_jobs=-1)

Block 1  score: 0.4347895817295355

Block 2  score: 0.4396651270207852

Block 3  score: 0.43918398768283295

Block 4  score: 0.4374198101103413

Block 5  score: 0.4384462406979728

Block 6  score: 0.43684244290479857

Block 7  score: 0.43687451886066203

Block 8  score: 0.4382858609186554

Result score: 0.43737445421412846
